# CommercialCompoundSearcher
An internal Sigman Lab tool for assessing the commercial availability of molecules based on the Pubchem database. We highly recommended to begin with a small subset (~25 molecules) to test the script first before using it on a larger dataset. Once a few variables are defined in later cells (like vendors to ignore), the script can be run autonomously by running all cells.


[https://github.com/thejameshoward/CommercialCompoundSearcher](https://github.com/thejameshoward/CommercialCompoundSearcher)

### Imports

In [1]:
# Built ins
import ast
import time
import urllib
import sqlite3
import urllib.error
from pathlib import Path
from pprint import pprint


# Data manipulation
import pandas as pd
import numpy as np

# Custom
from utils import canonicalize_smiles, smiles_to_inchi_key, smiles_to_inchi
from utils import remove_duplicate_inchi_keys
from utils import get_cid_from_inchi_key, get_vendor_list_from_cid
from utils import remove_specific_vendors_from_dataframe
from utils import draw_molecules_to_grid_image
from utils import convert_str_list
from utils import get_CAS_from_cid, get_SMILES_from_CAS, get_smiles_from_cid
from database import init_smiles_cache, get_cached_smiles, upsert_cached_smiles
from utils import PubchemVendor


## Reading in list of smiles

The list of smiles can be a plaintext document with __SMILES__ in the first line and the '.txt' extension. Alternatively, the file extension could be '.csv' and also contain a __SMILES__ column.

```
┌────────────────────────┐
│ SMILES                 │
│ CC(=O)OCC[N+](C)(C)C   │
│ CC(C[N+](C)(C)C)OC(=O) │
│ ...                    │
└────────────────────────┘

```


In [ ]:
# Define a file
file = Path('./data/cids.txt')

drop_empty_rows = False

# Read in the file
if file.suffix == '.txt':
    df = pd.read_table(file, header=0)
elif file.suffix == '.csv':
    df = pd.read_csv(file, header=0)
else:
    raise ValueError(f'{file.name} does not have a supported extension.')

# Check that the file is formatted correctly
if not 'SMILES' in df.columns and 'INCHI_KEY' not in df.columns:
    raise KeyError('The column "SMILES" or "INCHI_KEY" should be in your provided spreadsheet or table.')

# Drop any empty rows
if drop_empty_rows:
    df.dropna(axis=0, how='any', inplace=True)

display(df)

## Canonicalization and additional molecular identifiers

This section is used to canonicalize the SMILES and add additional molecular identifier information using RDKit. The output of this block will contain warnings (and potentially errors) from RDKit. Many of these errors (such as None mol from RDKit) are handled by removing the SMILES string and storing it in a separate file. 

In [ ]:
# Apply canonicalization
#TODO Understand how this affects stereoisomerism in the SMILES/InChI/InChI key values
df['SMILES'] = df['SMILES'].apply(canonicalize_smiles)

# Add InChI column
df['INCHI'] = df['SMILES'].apply(smiles_to_inchi)

# Add InChI key column
df['INCHI_KEY'] = df['SMILES'].apply(smiles_to_inchi_key)

# Get every row that has np.nan values
failed = df[(df['INCHI'].isna()) | (df['INCHI_KEY'].isna())]

# Get every row that does not have np.nan values
df = df[~(df['INCHI'].isna()) | ~(df['INCHI_KEY'].isna())].copy(deep=True)

# Save the failed and the canonicalized datasets to a csv file
failed.to_csv('./results/failed_canonicalization.csv', index=False)
df.to_csv('./results/canonicalized.csv', index=False)

# Check if anything failed an notify the user
if failed.empty:
    print('No SMILES strings failed canonicalization.')
else:
    display(failed)

display(df)

## Remove duplicate InChI keys

Because we will be using REST queries to gather vendor information, it is important to remove duplicates because they will "waste" and REST query. This procedure removes __exact__ duplicates of the InChI key in the dataframe even if the SMILES string is different.

In [ ]:
# Remove exact duplicates
df, duplicates = remove_duplicate_inchi_keys(df=df)

# For your viewing pleasure
display(df)

if duplicates.empty:
    print('No duplicate entries were found.')
else:
    display(duplicates)

# Save the results for good book keeping.
df.to_csv('./results/added_molecular_identifiers.csv', index=False)
duplicates.to_csv('./results/duplicate_molecular_identifiers.csv', index=False)

## Query Pubchem for CID

The best identifier to use for querying Pubchem is the Pubchem Compound ID (CID). For more information on how Pubchem standardizes its database, please see the [compounds webpage](https://pubchem.ncbi.nlm.nih.gov/docs/compounds). This section will obtain a CID for a given InChi key. The REST queries each take at least 200 ms.

In [ ]:
# Get inchi keys as a list
inchi_keys = df['INCHI_KEY'].to_list()

# Define a directory in which to store CID values
cid_dir = Path('./results/cids/')
cid_dir.mkdir(exist_ok=True)

# This assertion statement will fail if you have duplicate
# InChi keys. If you don't care, remove the following line
assert len(list(set(inchi_keys))) == df.shape[0]

# Get the total length of InChI keys for tracking progress
total = len(inchi_keys)

# Enumerate over all inchi keys and add CID values
for i, inchi_key in enumerate(inchi_keys):

    if Path(cid_dir / inchi_key).exists():
        print(f'Found {inchi_key} in {cid_dir.absolute()}')
        with open(cid_dir / inchi_key, 'r') as _:
            cid = _.read()
    else:
        print(f'Working on {i + 1} of {total} ({round((i + 1) / total * 100, 2)}%)')

        # Set cid to nan if we can't find it
        cid = np.nan

        # Try to get the CID, if there is no CID, skip
        # This sleep here seems to prevent connection resets by Pubchem
        time.sleep(0.01)
        try:
            cid = get_cid_from_inchi_key(inchi_key)
            with open(cid_dir / inchi_key, 'w') as _:
                _.write(str(cid))
        except urllib.error.HTTPError as e:
            print(f'Could not convert InChi Key {inchi_key} to CID because {e}. Skipping.')
            continue
        except Exception as e:
            print(f'Could not convert InChi Key {inchi_key} to CID because {e}. Skipping.')
            continue

    # Check how many instances of that INCHI_KEY are in the df
    if df[df['INCHI_KEY'] == inchi_key].shape[0] != 1:
        print(f'WARNING: Found more than one InChI key {inchi_key}!')

    # Add the CID based on inchi_key
    df.loc[df['INCHI_KEY'] == inchi_key, 'CID'] = str(cid)

# Get the df of molecules for which there is no CID, save it for good book keeping
no_cids = df[df['CID'].astype(float).isna()]
no_cids.to_csv('./results/no_cid_found.csv', index=False)

# Get the new df that has CID values for each molecule, save it for for records
df = df[~(df['CID'].astype(float).isna())].copy(deep=True)
df.to_csv('./results/added_cid.csv', index=False)

display(df)

## Query Pubchem for vendors

This section will us the CID values found in the previous cell to acquire a list of vendors from Pubchem. The REST queries each take at least 200 ms.

In [ ]:
df = pd.read_csv('./results/added_cid.csv', header=0)

# Get inchi keys as a list
cids = df['CID'].astype(int).to_list()

# Define a directory in which to store vendor information
vendor_dir = Path('./results/vendors/')
vendor_dir.mkdir(exist_ok=True)

# This assertion statement will fail if you have duplicate
# InChi keys. If you don't care, remove the following line
assert len(list(set(cids))) == df.shape[0]

# Get the total number of CIDs for tracking progress
total = len(cids)

# Keep a list of CIDs that have no vendors
no_vendor_cids = []

# Make a CID vendor dictionary that will contain the
# PubchemVendor objects
cid_vendor_dict = {}

# Enumerate over all CIDs and look for vendors
for i, cid in enumerate(cids):

    # Name the file for storing the vendor data
    vendor_file = Path(vendor_dir / f'{cid}.txt')

    # If the vendor information file already exists, read it in as a list of PubchemVendor instances
    if vendor_file.exists():
        print(f'Found {cid} in {vendor_dir.absolute()}')
        with open(vendor_file, 'r') as _:
            vendors = [PubchemVendor(cid, x) for x in ast.literal_eval(_.read())]
    else:
        print(f'Working on {i + 1} of {total} ({round((i + 1) / total * 100, 2)}%)')

        # Try to get the list of PubchemVendor objects
        try:
            vendors = list(set(get_vendor_list_from_cid(cid)))
            with open(vendor_file, 'w', encoding='utf-8') as _:
                _.write(str([v.vendor_info for v in vendors]))
        except urllib.error.HTTPError as e:
            print(f'Could not get vendor list from CID {cid}.')
            no_vendor_cids.append(int(cid))
            continue

    # Check how many instances of that CID are in the df
    if df[df['CID'].astype(int) == cid].shape[0] != 1:
        print(f'WARNING: Found more than one CID for {cid}!')

    # Add the CID/VENDORS based on inchi_key
    df.loc[df['CID'].astype(int) == cid, 'VENDORS'] = str([x.SourceName for x in vendors])

    # Add the CID:Vendor key:value pair
    cid_vendor_dict[cid] = vendors

# Get the df of molecules for which there are no vendors, save it for good book keeping
no_vendors = df[df['CID'].astype(int).isin(no_vendor_cids)].copy(deep=True)
no_vendors.to_csv('./results/no_vendors_found.csv')

# Get all the molecules that have vendors
df = df[~df['CID'].astype(int).isin(no_vendor_cids)]
df.to_csv('./results/with_vendors_found.csv', index=False)

display(df)

#print(cid_vendor_dict)

## Filtering Vendors

The term "commercial availability" may differ between applications. Some vendors report that a compound is purchasable but will only synthesize it upon request. Additionally, the geographic location of the vendor's warehouse may lead to extended shipping times. In this section, we can filter vendors by selecting them from a list of total vendors.

The next cells are organized into separate steps.

In [ ]:
# Print the total list of vendors
df = pd.read_csv('./results/with_vendors_found.csv')
list_of_current_vendors = list(set([vendor for vendor_list in df['VENDORS'].apply(convert_str_list) for vendor in vendor_list]))
display(f'UNIQUE VENDORS:')
pprint(list_of_current_vendors)
print(f'\nN_UNIQUE_VENDORS: {len(list_of_current_vendors)}')


#### Select vendors to keep __OR__ vendors to remove

Two variables are declared below. Define one and only one of these variables to be a list of vendor strings. __This section relies on exact string comparison. Thus, it is important that the **exact** string is used from the block above.__ We recommend using VENDORS_TO_REMOVE to be more deliberate with vendor selection.

(experimental) We've included a list of vendors as a template for VENDORS_TO_KEEP.

In [ ]:
# Define only one of these as a list
VENDORS_TO_KEEP = ['TCI (Tokyo Chemical Industry)',
 'Ambeed',
 'Combi-Blocks',
 'Thermo Fisher Scientific',
 'Sigma-Aldrich',
 'VWR, Part of Avantor']

VENDORS_TO_REMOVE = None

# Convert the string representation of the list of vendors
# to an actual Python list
df['VENDORS'] = df['VENDORS'].apply(convert_str_list)

# Get the list of current vendors (again)
list_of_current_vendors = list(set([vendor for vendor_list in df['VENDORS'].apply(convert_str_list) for vendor in vendor_list]))

# Convert vendors to keep into a vendors_to_remove list
if VENDORS_TO_REMOVE is None and VENDORS_TO_KEEP is not None:
    VENDORS_TO_REMOVE = [x for x in list_of_current_vendors if x not in VENDORS_TO_KEEP]

# Illegal options
elif VENDORS_TO_REMOVE is not None and VENDORS_TO_KEEP is not None:
    raise ValueError(f'Define either VENDORS_TO_REMOVE or VENDORS_TO_KEEP as a list not both.')

# User not removing any vendors
elif VENDORS_TO_REMOVE is None and VENDORS_TO_KEEP is None:
    VENDORS_TO_REMOVE = []

else:
    raise ValueError(f'Make sure you define the unused variable at the beginning of this cell to None')

# Remove the unwanted vendors
df = remove_specific_vendors_from_dataframe(df, vendors=VENDORS_TO_REMOVE)

# Purge empty df entries now
#TODO Reevaluate use of string literals as list intermediates
df = df[~(df['VENDORS'].astype(str) == '[]')]

display(df)


'''
# TODO Get the links from the vendor information from pubchem
# Get the new list of vendors
list_of_current_vendors = list(set([vendor for vendor_list in df['VENDORS'].to_list() for vendor in vendor_list]))
print(f'UNIQUE VENDORS:')
pprint(list_of_current_vendors)
print(f'\nN_UNIQUE_VENDORS: {len(list_of_current_vendors)}')

display(df)

for col in df.columns:
    print(df[col].dtype)
'''

#### Save the curated list of molecules

In [ ]:
list_of_current_vendors = list(set([vendor for vendor_list in df['VENDORS'].apply(convert_str_list) for vendor in vendor_list]))

# Add a link for each current vendor
for v in list_of_current_vendors:
    df[f'{v}_link'] = 'NONE'

rows = []
for i, row in df.iterrows():
    # For each vendor that sells that row
    for vendor_name in row['VENDORS']:
        all_vendor_objects = cid_vendor_dict[row['CID']]
        all_vendor_objects = [x for x in all_vendor_objects if x.SourceName == vendor_name]
        if len(all_vendor_objects) == 0:
            continue
        row[f'{vendor_name}_link'] = all_vendor_objects[0].SourceRecordURL
    rows.append(row)

df = pd.DataFrame(rows)
display(df)

# Save the
df.to_csv('./results/with_vendor_links.csv', index=False)

In [ ]:
df.to_csv('./FINAL_LIBRARY_CURATED.csv', index=False)

## Query Pubchem for CAS number

This section will us the CID values found in the previous cells to acquire a CAS number from Pubchem. Often a molecule will have multiple CAS numbers and the `get_CAS_from_CID()` function will gather only one of them. Users should be aware that this looks for two dashes (-) in the numbers it receives from Pubchem to identify the CAS number. This procedure could be improved by a more systematic way of determining whether the item received from Pubchem is actually a CAS number. The REST queries each take at least 200 ms.

__If you stop this cell while it is running, you will lose all of your progress towards acquiring vendors__

In [ ]:
# Get the full list of CIDs from the library
cids = [int(x) for x in df['CID'].to_list() if x != '']

# Get the total number of CIDs for tracking progress
total = len(cids)

# Keep a list of CIDs that have no vendors
no_vendor_cids = []

# Enumerate over all CIDs and look for vendors
for i, cid in enumerate(cids):
    print(f'Working on {i + 1} of {total} ({round((i + 1) / total * 100, 2)}%)')

    # Try to get the list of PubchemVendor objects
    try:
        cas = get_CAS_from_cid(cid)
    except urllib.error.HTTPError as e:
        print(f'Could not get CAS number from CID {cid}. ERROR: {e}')
        continue

    # Add the CID/VENDORS based on inchi_key
    df.loc[df['CID'].astype(int) == cid, 'CAS_NUMBER'] = str(cas)

    # Check how many instances of that CID are in the df
    #if df[df['CID'].astype(int) == cid].shape[0] != 1:
    #    print(df[df['CID'] == cid])
    #    print(f'WARNING: Found more than one CID for {cid}!')
    ## Add the CID/VENDORS based on inchi_key
    #df.loc[df['CID'].astype(int) == cid, 'CAS'] = str(cas)

# Save the file
df.to_csv('./FINAL_LIBRARY_CURATED_with_cas_numbers.csv', index=False)



## Query a Sigman Inventory Export for the CAS numbers

Export a full copy of the Sigman inventory in labsuit and point the inventory_spreadsheet variable to its path. This cell will create a slice of the dataframe that contains CAS numbers in both your curated library and the inventory spreadsheet.

In [ ]:
# Define the spreadsheet file
inventory_spreadsheet = Path('./data/Sigman-inventory-03-07-2024-example.xlsx')

# Read in the file (These settings should read the default format)
try:
    inventory = pd.read_excel(file, header=0, sheet_name='Chemical', engine='openpyxl')
except Exception: # This is required because I think labsuit is not zipping their xlsx files
    with open(inventory_spreadsheet, 'rb') as infile:
        inventory = pd.read_excel(infile, sheet_name='Chemical')

# Filter inventory by presence of CAS
inventory['CAS_NUMBER'] = inventory['CAS_NUMBER'].astype(str).apply(str.strip)
inventory = inventory[inventory['CAS_NUMBER'].astype(str).isin(df['CAS_NUMBER'])]

display(inventory)

# Save the owned molecules in the results folder
inventory.to_csv('./results/owned_molecules.csv', index=False)

## Query Pubchem for SMILES from CAS Number
As written, the dataframe created from your spreadsheet must have the column name `CAS_NUMBER` somewhere in it

In [ ]:

# Define the spreadsheet file
inventory_spreadsheet = Path('./haruka/CBr-inventory-01-29-2026.xlsx')
sheet_name = 'Chemical'

# Define a directory in which to store information
smiles_from_cas_dir = Path('./haruka/smiles_from_cas_raw_files/')
smiles_from_cas_dir.mkdir(exist_ok=True)

# Read in the file (These settings should read the default format)
if inventory_spreadsheet.suffix == '.xlsx':
    try:
        inventory = pd.read_excel(file, header=0, sheet_name=sheet_name, engine='openpyxl')
    except Exception: # This is required because I think labsuit is not zipping their xlsx files
        with open(inventory_spreadsheet, 'rb') as infile:
            inventory = pd.read_excel(infile, sheet_name=sheet_name)
else:
    inventory = pd.read_csv(inventory_spreadsheet, header=0)

# Get the full list of CAS from the library
CAS_NUMBERS = [str(x) for x in inventory['CAS_NUMBER'].to_list() if x != '']

# Get the total number of CAS for tracking progress

total = len(CAS_NUMBERS)
print(f'There are {total} CAS numbers to process.')
print(f'There are {len(list(set(CAS_NUMBERS)))} unique CAS numbers to process.')
#from collections import Counter
#pprint(Counter(CAS_NUMBERS))
#assert False

# Keep a list of CAS numbers for which we could get no SMILES
no_vendor_cids = []

# Enumerate over all CAS and look for vendors
for i, CAS in enumerate(CAS_NUMBERS):
    print(f'Working on {i + 1} of {total} ({round((i + 1) / total * 100, 2)}%)')

    cas_file = Path(smiles_from_cas_dir / f'{CAS}.txt')

    if cas_file.exists():

        with open(cas_file, 'r') as _:
            smiles = str(_.read())

        print(f'Found {CAS} in {cas_file.absolute()}\t{smiles}')
    else:

        # Try to get the list of PubchemVendor objects
        try:
            smiles = get_SMILES_from_CAS(CAS)
            with open(cas_file, 'w', encoding='utf-8') as o:
                o.write(smiles)
        except urllib.error.HTTPError as e:
            print(f'Could not get SMILES string from CAS {CAS}. ERROR: {e}')
            continue

    # Add the CID/VENDORS based on inchi_key
    inventory.loc[inventory['CAS_NUMBER'].astype(str) == CAS, 'SMILES'] = str(smiles)

# Save the file
inventory.to_excel('./haruka/CBr-inventory-01-29-2026_with_smiles.xlsx', index=False)



There are 727 CAS numbers to process.
There are 546 unique CAS numbers to process.
Working on 1 of 727 (0.14%)
Found 506-96-7 in /Users/jameshoward/Documents/Programming/CommercialCompoundSearcher/haruka/smiles_from_cas_raw_files/506-96-7.txt	CC(=O)Br
Working on 2 of 727 (0.28%)
Found 74-96-4 in /Users/jameshoward/Documents/Programming/CommercialCompoundSearcher/haruka/smiles_from_cas_raw_files/74-96-4.txt	CCBr
Working on 3 of 727 (0.41%)
Found 109-64-8 in /Users/jameshoward/Documents/Programming/CommercialCompoundSearcher/haruka/smiles_from_cas_raw_files/109-64-8.txt	C(CBr)CBr
Working on 4 of 727 (0.55%)
Found 106-94-5 in /Users/jameshoward/Documents/Programming/CommercialCompoundSearcher/haruka/smiles_from_cas_raw_files/106-94-5.txt	CCCBr
Working on 5 of 727 (0.69%)
Found 86-90-8 in /Users/jameshoward/Documents/Programming/CommercialCompoundSearcher/haruka/smiles_from_cas_raw_files/86-90-8.txt	C1=CC2=C(C=C1Br)C(=O)OC2=O
Working on 6 of 727 (0.83%)
Found 106-93-4 in /Users/jameshoward

## Query Pubchem for SMILES

This section will us the CID values found in the previous cell to acquire the canonical SMILES Pubchem. The REST queries each take at least 200 ms.

In [ ]:
# Read in the CIDs file
df = pd.read_table('./data/cids_small.txt', header=0)

# Make sure CID is present, integer, and use it as the dataframe index
df = df.dropna(subset=['CID']).copy()
df['CID'] = df['CID'].astype(int)
df = df.set_index('CID', drop=True)

# Ensure SMILES column exists
if 'SMILES' not in df.columns:
    df['SMILES'] = None

# Make a list of CIDs to process
cids = df.index.unique().tolist()
total = len(cids)

# Initialize SQLite cache
db_path = Path('./results/cache/smiles_cache.sqlite')

commit_every = 100
writes_since_commit = 0

# Initialize SQLite cache and make the connection
with init_smiles_cache(db_path) as conn:
    for i, cid in enumerate(cids, start=1):

        # Try cache first
        smiles, status = get_cached_smiles(conn=conn, cid=cid)

        if status == 'ok' and smiles:
            # Cached successful result
            df.loc[cid, 'SMILES'] = smiles
            continue

        if status == 'missing':
            # Store in the DataFrame as None so that it's empty cell
            df.loc[cid, 'SMILES'] = None
            continue

        # Do nothing (retry on http_error)
        if status not in ['ok', 'missing', 'http_error', 'parse_error']:
            raise ValueError(f'Unexpected cache status "{status}"')

        print(f'Working on {i} of {total} ({round(i / total * 100, 2)}%)')

        try:
            smiles = get_smiles_from_cid(cid)
        except urllib.error.HTTPError as e:
            # Cache the failure so you can decide later whether to retry
            upsert_cached_smiles(conn, cid, smiles='', status='http_error')
            writes_since_commit += 1
            print(f'Could not get SMILES from CID {cid}. ERROR: {e}')
            continue

        # Normalize / handle empty results
        if smiles is None or str(smiles).strip() == '':
            upsert_cached_smiles(conn, cid, '', 'missing')

            # Store in the DataFrame as None so that it's empty cell
            df.loc[cid, 'SMILES'] = None

            writes_since_commit += 1
        else:
            smiles = str(smiles).strip()
            upsert_cached_smiles(conn, cid, smiles, 'ok')
            df.loc[cid, 'SMILES'] = smiles
            writes_since_commit += 1

        # Commit periodically for speed + crash-safety
        if writes_since_commit >= commit_every:
            conn.commit()
            writes_since_commit = 0

    # Final commit
    conn.commit()

# Write to CSV
df.to_csv('./data/cids_with_smiles_james.csv')
print(df)


                                                      SMILES
CID                                                         
18676105                         CC(=O)CC(=O)C1=CC(=NC=C1)Cl
24986779                                  CC(C(=O)CC(=O)C)OC
69175241                        CC1(CCC(C1=O)C(=O)C(F)(F)F)C
43186929                     CC1=CC=C(O1)C(=O)CC(=O)C(F)(F)F
255149                    CC1=CC(=C(O1)C)C(=O)CC(=O)C(F)(F)F
117066419            CC(=O)C1C(=O)C2=CC=CC=C2S(=O)(=O)N1CC#N
117066154      C1CCC(=O)C(C1)C(=O)C2=CC=C(C=C2)OCC3=CC=CC=C3
2795191            COC(=O)C(=O)CC(=O)C1=CC=CC=C1[N+](=O)[O-]
5043406          CC1=CC=C(C=C1)C(=O)CCC2C(=O)C3=CC=CC=C3C2=O
117064303  CC=NN=C1C(C(=O)C2=CC=CC=C21)C(=O)C(C3=CC=CC=C3...
117064317  C1CCC(=NN=C2C(C(=O)C3=CC=CC=C32)C(=O)C(C4=CC=C...
1480007                CC(=O)C(CC1=CC(=CC=C1)C(F)(F)F)C(=O)C
45055342              COC(=O)C(=O)CC(=O)C1=CC(=C(C=C1Cl)Cl)F
43118997                  COC(=O)C(=O)CC(=O)C1=CC=C(C=C1)C#N
4063828                 

## Drawing molecules 🥳 !

In this section we've included some useful functions for drawing molecules in your library.

In [ ]:
# Get a list of all smiles
smiles = df['SMILES'].to_list()

print(f'Number of SMILES: {len(smiles)}')

smiles = smiles[:1000]

# Get the PIL images of the grid by passing smiles list
images = draw_molecules_to_grid_image(smiles, mols_per_row=6, img_resolution=600)

for image in images:
    display(image)